# Assignment — Deep Recurrent Graph Generation

### Task 1. Recurrent attention GNN (1 point)

In [4]:
#!pip install dgl -f https://data.dgl.ai/wheels/repo.html -q
!pip install dgl-cu113 -f https://data.dgl.ai/wheels/repo.html -q
import dgl
dgl.__version__

'0.7.2'

In [5]:
import dgl
import dgl.nn as dnn
import dgl.function as fn
from dgl.data import DGLDataset, GINDataset

import torch
import torch.nn as nn
import torch.distributions as D
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ks_2samp

from tqdm.notebook import trange, tqdm
import requests

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

The traditional graph generation approaches (Erdos-Renyi, Barabasi-Albert, etc.) can be used to efficiently generate synthetic graphs that have certain properties. Such models can give us insight into how certain graph structures might arise in the real world. However, they rely on a fixed, handcrafted generation process. On the other side, deep generation approaches will seek to learn a generative model on a set of training graphs. This assignment is dedicated to a such generative model inspired by GRAN [Liao et al., 2019](https://arxiv.org/pdf/1910.00760.pdf). 

The idea of the model is to generate a graph by adding nodes one-by-one. Features of existing nodes are fed into a recurrent attention GNN for creating node embedding. Then the embeddings used for estimating parameters of Bernoulli distributions that can model probabilities of new edges. First, let us create a recurrent attention model for node embedding. Each layer in the network takes a graph with node states $h^l$ and propagate them as follows:

$$h^{l+1}_i = \text{GRU}\left(h^l_i, \sum_{j\in\mathcal{N}(i)} a_{ij}^l m_{ij}^l\right)$$
$$m_{ij}^l = f(h_i^l - h_j^l)$$
$$a_{ij}^l = g(h_i^l - h_j^l)$$

where $\mathcal{N}(i)$ is a neighborhood of the node $i$, $f(\cdot)$ is the message function, $g(\cdot)$ is the attention head (both are two-layer MLP). The initial node states are node features. The node embeddings are final node states.

Write a function `_prop` that takes a graph, node states, index of a layer and propagate current node states, returns next node states.

*Hints:* 
* *apply the DGL message passing framework*
* *`fn.u_sub_v` for creating messages — difference between adjacent node states*
* *`graph.apply_edges` for storing the messages into edge features*
* *`graph.update_all` for reduction messages from adjacent edges (summation in this task)*

In [21]:
class RecurrentGNN(nn.Module):
    def __init__(self, node_state_dim, hid_dim, num_layer):
        super().__init__()
        self.num_layer = num_layer
        self.update_func = nn.ModuleList([
            nn.GRUCell(input_size=hid_dim, hidden_size=node_state_dim) 
            for _ in range(self.num_layer)
        ])
        self.msg_func = nn.ModuleList([
            nn.Sequential(
                *[
                    nn.Linear(node_state_dim, hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim, hid_dim)
                ]) for _ in range(self.num_layer)
        ])
        self.att_head = nn.ModuleList([
            nn.Sequential(
                *[
                    nn.Linear(node_state_dim, hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim, hid_dim),
                    nn.Sigmoid()
                ]) for _ in range(self.num_layer)
        ])

    def _prop(self, graph, state, layer_idx=0):
        graph.ndata['state'] = state
        graph.apply_edges(fn.u_sub_v('state', 'state', 'sub'))

        graph.edata['f'] = self.msg_func[layer_idx](graph.edata['sub'])
        graph.edata['g'] = self.att_head[layer_idx](graph.edata['sub'])

        graph.edata['mult'] = graph.edata['f'] * graph.edata['g']    
        
        graph.update_all(fn.copy_e('mult', 'msg'), fn.sum('msg', 'res'))
            
        output = self.update_func[layer_idx](graph.ndata['res'], graph.ndata['state'])

        return output

    def forward(self, graph, feat):
        state = feat
        for i in range(self.num_layer):
            if i > 0:
                state = F.relu(state)
            state = self._prop(graph, state, layer_idx=i)
        return state

In [23]:
torch.manual_seed(0)
model = RecurrentGNN(node_state_dim=16, hid_dim=128, num_layer=5)
graph = dgl.from_networkx(nx.grid_2d_graph(2, 2))
feat = torch.ones(4, 16)

model._prop(graph, feat, layer_idx=1)

tensor([[0.6579, 0.4505, 0.7249, 0.5690, 0.5839, 0.2217, 0.7943, 0.6908, 0.7634,
         0.1489, 0.5663, 0.6020, 0.6967, 0.7424, 0.1602, 0.4054],
        [0.6579, 0.4505, 0.7249, 0.5690, 0.5839, 0.2217, 0.7943, 0.6908, 0.7634,
         0.1489, 0.5663, 0.6020, 0.6967, 0.7424, 0.1602, 0.4054],
        [0.6579, 0.4505, 0.7249, 0.5690, 0.5839, 0.2217, 0.7943, 0.6908, 0.7634,
         0.1489, 0.5663, 0.6020, 0.6967, 0.7424, 0.1602, 0.4054],
        [0.6579, 0.4505, 0.7249, 0.5690, 0.5839, 0.2217, 0.7943, 0.6908, 0.7634,
         0.1489, 0.5663, 0.6020, 0.6967, 0.7424, 0.1602, 0.4054]],
       grad_fn=<AddBackward0>)

In [24]:
torch.manual_seed(0)
model = RecurrentGNN(node_state_dim=16, hid_dim=128, num_layer=5)
graph = dgl.from_networkx(nx.grid_2d_graph(2, 2))
feat = torch.ones(4, 16)
out = model._prop(graph, feat, layer_idx=0)
assert out.shape == (4, 16)
test_ans = out.detach().numpy()[0, :3].round(2)
assert np.all(np.isclose(test_ans, [0.68, 0.57, 0.29]))
out = model._prop(graph, feat, layer_idx=1)
test_ans = out.detach().numpy()[0, :3].round(2)
assert np.all(np.isclose(test_ans, [0.66, 0.45, 0.72]))

### Task 2. Bernoulli probabilistic model (1 point)

Next, the generation process is performed as follows: 
1. Start from a graph with a single node
2. Produce parameters of the distributions based on graph
3. Add new node to the graph and sample edges from new node using the distributions
4. Repeat steps 2-3 to the desired number of nodes

<img src='https://raw.githubusercontent.com/netspractice/advanced_gnn/main/assignment_recurrent_generation/gnn_generation.png' width=600>

For example, we have the graph with 4 nodes, we produce 4 parameters of 4 independent Bernoulli distributions, and then sample 4 boolean values. Let it be (0, 0, 1, 1), then we connect the new node to the third and fourth nodes. In this assignment, let us use MLP for converting node embeddings into 1-dimensional parameters of Bernoulli distributions.

Write a class `BernoulliGNN`. The network takes a graph and node features, propagate them through a recurrent GNN, then through a two-layer MLP and returns parameters of distributions.

In [25]:
class BernoulliGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, num_layer):
        super().__init__()
        self.hid_dim = hid_dim
        self.gru = RecurrentGNN(node_state_dim=in_dim, hid_dim=hid_dim, num_layer=num_layer)
        self.mlp = nn.Sequential(nn.Linear(in_dim, in_dim), 
                                 nn.Linear(in_dim, 1), 
                                 nn.Sigmoid()
                                )
        
    def forward(self, graph, feat):
        x = self.gru(graph, feat)
        out = self.mlp(x)       
        
        return out

In [ ]:
model = BernoulliGNN(in_dim=16, hid_dim=128, num_layer=5)
graph = dgl.from_networkx(nx.grid_2d_graph(2, 2))
feat = torch.ones(4, 16)
theta = model(graph, feat)
theta

tensor([[0.5090],
        [0.5090],
        [0.5090],
        [0.5090]], grad_fn=<SigmoidBackward>)

In [26]:
model = BernoulliGNN(in_dim=16, hid_dim=128, num_layer=5)
graph = dgl.from_networkx(nx.grid_2d_graph(2, 2))
feat = torch.ones(4, 16)
theta = model(graph, feat)
assert theta.shape == (4, 1)
assert torch.all((0 <= theta) & (theta <= 1))

Write a function `sample` that takes parameters `theta` and returns realization from corresponding Bernoulli distributions.

In [27]:
def sample(theta):
    return torch.bernoulli(theta)

In [28]:
theta = torch.tensor([[0.999], [0.001], [0.999], [0.999]])
assert torch.all(sample(theta) == torch.tensor([[1], [0], [1], [1]]))

### Task 3. Generation process (1 point)

Let us check what we can generate using an untrained model. Let the node features be one-hot encoded vector with node index. The node features dimension can be larger than the number of nodes, so let it be padded with zeros on the right. For example, 6-dimensional node features in a graph with 4 nodes be:

$$F = \begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
\end{pmatrix}$$


Write a function `generate_graph` that takes a model, number of nodes, node features dimension and returns a generated graph and node features. The generated graph is undirected and contains self-loops.

In [29]:
def generate_graph(model, n_nodes, feat_dim):
    # starting from a graph with a single node
    feat = torch.zeros(1, feat_dim).to(device)
    feat[0, 0] = 1
    graph = dgl.graph(data=([0], [0])).to(device)
    for i in range(1, n_nodes):
        nodes = graph.nodes()
        src_nodes, dst_nodes = graph.edges()
        theta = model(graph, feat)
        new_edges = sample(theta).to(torch.bool).squeeze()
        
        new_src_nodes = nodes[new_edges]
        
        new_src_nodes = torch.cat((new_src_nodes.view(-1), torch.tensor([i])))
        src_nodes = torch.cat((src_nodes, new_src_nodes)).squeeze()
        
        new_dst_nodes = torch.tensor([i] * new_src_nodes.shape[0])
        dst_nodes = torch.cat((dst_nodes, new_dst_nodes))
        
        src_nodes = torch.cat((src_nodes, new_dst_nodes[:-1]))
        dst_nodes = torch.cat((dst_nodes, new_src_nodes[:-1]))
        
        graph = dgl.graph(data=(src_nodes, dst_nodes)).to(device)
        
        pad_with = feat_dim - graph.number_of_nodes()
        feat = F.pad(torch.diag(torch.ones(graph.number_of_nodes())), (0, pad_with), 'constant', 0)
                
    return graph, feat

In [ ]:
model = BernoulliGNN(in_dim=16, hid_dim=128, num_layer=5)
model = model.to(device)
graph, feat = generate_graph(model, n_nodes=16, feat_dim=16)
graph, feat = graph.cpu(), feat.cpu()

In [31]:
model = BernoulliGNN(in_dim=16, hid_dim=128, num_layer=5)
model = model.to('cpu')
graph, feat = generate_graph(model, n_nodes=16, feat_dim=16)
graph, feat = graph.cpu(), feat.cpu()

assert graph.number_of_nodes() == 16
assert graph.number_of_edges() > 1
assert feat.shape == (16, 16)
assert torch.all(feat[:3, :3] == torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]]))
adj = graph.adj().to_dense()
assert torch.all(adj[range(10), range(10)] == 1)
assert torch.all(adj == adj.t())

DGLError: ignored

In [ ]:
G = nx.Graph(graph.to_networkx())
G.remove_edges_from(nx.selfloop_edges(G))
nx.draw_kamada_kawai(G)

### Task 4. Negative log likelihood loss (1 point)

We aim to train the model so that it can be able to generate realistic networks. Fortunately, explicit parametric models can be optimized by maximim likelihood estimation. In torch, it can be performed by minimization negative log likelihood loss.

Write a function `bernoulli_nll` that takes parameters `theta` and a realization `label`, returns the mean negative log likelihood.

In [ ]:
def bernoulli_nll(theta, label):
    
    return -(torch.log(theta) * label + torch.log(1 - theta) * (1 - label)).mean()

In [ ]:
theta = torch.tensor([[0.999], [0.001], [0.999], [0.999]])
label = torch.tensor([[1.], [0.], [1.], [1.]])
loss1 = bernoulli_nll(theta, label)
label = torch.tensor([[1.], [0.], [0.], [1.]])
loss2 = bernoulli_nll(theta, label)
label = torch.tensor([[0.], [1.], [0.], [0.]])
loss3 = bernoulli_nll(theta, label)
assert loss1 < loss2 < loss3

### Task 5. Dataloader for grid subgraphs (1 point)

Let us train the model to generate 2d grid graphs, it can be useful for visual verification of the model. The auxiliary function `grid_graphs` generates grid graphs with one-hot encoded node features and returns a list of DGL graphs.

In [ ]:
def grid_graphs(min_dim, max_dim, n_graphs, feat_dim):
    graphs = []
    for i in range(n_graphs):
        i = np.random.randint(min_dim, max_dim + 1)
        j = np.random.randint(min_dim, max_dim + 1)
        G = nx.grid_2d_graph(i, j)
        graph = dgl.from_networkx(G)
        graph = graph.add_self_loop()
        feat = torch.eye(graph.number_of_nodes())
        feat = F.pad(feat, [0, feat_dim - graph.number_of_nodes(), 0, 0])
        graph.ndata['feat'] = feat
        graphs.append(graph)
    return graphs

In [ ]:
nx.draw(grid_graphs(4, 4, 1, 16)[0].to_networkx())

Next, we need to define the train dataset where graphs are subgraphs from initial grid graphs and labels are next node indicator vectors. For a given graph, the label defines connections to the next node. For example, let the left depicted graph be a subgraph and the next node be 5, then a label be a column-vector `[[0], [0], [1], [1]]`.

<img src='https://raw.githubusercontent.com/netspractice/advanced_gnn/main/assignment_recurrent_generation/gnn_generation.png' width=600>

Write a function `process` that divides each graph into N subgraphs and prepares N labels where N is the number of nodes minus one. Then it stores all subgraphs and labels into `self.graphs` and `self.labels` lists.

For example, the dataset on the single graph with 5 nodes looks like:
* subgraph with nodes [0], label for the node 1
* subgraph with nodes [0, 1], label for the node 2
* subgraph with nodes [0, 1, 2], label for the node 3
* subgraph with nodes [0, 1, 2, 3], label for the node 4

In [ ]:
class GridDataset(DGLDataset):
    def __init__(self, feat_dim, min_dim, max_dim, n_graphs):
        self.feat_dim = feat_dim
        self.min_dim = min_dim
        self.max_dim = max_dim
        self.n_graphs = n_graphs
        self.graphs = None
        self.labels = None
        super().__init__(name='grid')

    def process(self):
        graphs = []
        labels = []
        for graph in grid_graphs(self.min_dim, self.max_dim, self.n_graphs, self.feat_dim):
            adj = graph.adj().to_dense()
            for i in range(1, graph.number_of_nodes()):
                graphs.append(graph.subgraph(range(i)))
                subgraph_labels = torch.tensor([k for k in adj[:i + 1, :i][-1]]).reshape(-1, 1)
                labels.append(subgraph_labels)
        self.graphs = graphs
        self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.labels[idx]

In [ ]:
feat_dim = 40
min_dim = 3
max_dim = 6
n_graphs = 100
grid_dataset = GridDataset(feat_dim, min_dim, max_dim, n_graphs)
assert n_graphs < len(grid_dataset) < n_graphs * max_dim**2
n_nodes = torch.tensor([g.number_of_nodes() for g, l in grid_dataset])
assert n_nodes.min() == 1
assert n_nodes.max() == max_dim**2 - 1
g, l = grid_dataset[1]
assert l.shape == (2, 1)
g, l = grid_dataset[2]
assert l.shape == (3, 1)
assert np.all([g.number_of_nodes() == l.shape[0] for g, l in grid_dataset])

In [ ]:
len(grid_dataset)

1977

Finally, create a train dataloader.

In [ ]:
def collate(sample):
    graphs, labels = map(list, zip(*sample))
    graph = dgl.batch(graphs)
    labels = torch.cat(labels)
    return graph, labels

grid_dataset = GridDataset(feat_dim=40, min_dim=4, max_dim=6, n_graphs=20)
dataloader = DataLoader(
    grid_dataset, batch_size=128, collate_fn=collate, shuffle=True)

for g, l in dataloader:
    break
len(dataloader), g.batch_num_nodes(), g.number_of_nodes(), l.shape

(5,
 tensor([ 8, 22, 11, 16, 11, 29, 13, 32, 12,  7, 21, 26,  4, 21, 12, 24, 27,  3,
         33, 17,  1, 10,  5,  8,  1, 24, 22, 13, 17, 16, 15, 26, 15,  3,  8, 17,
         27, 34, 17,  8,  9,  2,  3, 24,  2, 22, 15, 22,  3,  3,  6, 24,  8, 23,
         19, 20,  8, 19,  5,  1, 23, 17, 16,  3, 20, 16,  5, 17, 25, 16, 13,  7,
          3, 13,  4,  7,  9,  4, 20,  1, 14, 29, 11,  5, 27, 31, 21,  1,  1,  6,
          4, 18, 25, 17, 17, 24, 20,  9,  5, 32, 28, 17,  4, 13, 23, 11, 10, 24,
          8,  8, 18,  4, 12, 20,  1, 28, 21, 11, 11, 16, 24, 11, 30, 15, 14,  3,
         13, 13]),
 1841,
 torch.Size([1841, 1]))

### Task 6. Mini-batch training process (1 point)

Write a finction `train`. Here is a simple training process: calculate the NLL loss and make an optimization step.

In [ ]:
def train(model, dataloader, opt):
    loss_values = []
    for graph, label in dataloader:
        opt.zero_grad()
        logits = model(graph, graph.ndata['feat'])
        loss = bernoulli_nll(logits, label)
        loss.backward()
        opt.step()
        loss_values.append(loss)
    return sum(loss_values) / len(loss_values)

In [ ]:
model = BernoulliGNN(in_dim=40, hid_dim=128, num_layer=5)
model.to(device);

The training process takes about 10 minutes in Colab on CPU and about a minute in Colab on GPU.

In [ ]:
opt = Adam(model.parameters(), lr=0.0005)

n_epochs = 500
for e in trange(n_epochs):
    loss = train(model, dataloader, opt)
    if e % 50 == 0 or e+1 == n_epochs:
        print(f'Epoch: {e+1}/{n_epochs}, NLL loss: {loss:.4f}')
        #plt.figure(figsize=(2,2))
        graph, feat = generate_graph(model, n_nodes=16, feat_dim=40)
        G = nx.Graph(graph.cpu().to_networkx())
        G.remove_edges_from(nx.selfloop_edges(G))
        #nx.draw_kamada_kawai(G, node_size=30)
        #plt.show()

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 1/500, NLL loss: 0.6723


Let us generate a few graphs from the model. They should be visually close to grid graphs.

In [ ]:
plt.figure(figsize=(10, 10))
for j in range(9):
    plt.subplot(3, 3, j+1)
    graph, feat = generate_graph(model, n_nodes=18, feat_dim=40)
    G = nx.Graph(graph.cpu().to_networkx())
    G.remove_edges_from(nx.selfloop_edges(G))
    nx.draw_kamada_kawai(G, node_size=30)

The next cell checks that at least one graph in a sample is close to a grid. It is possible with approximately 0.05 NLL loss. Note that all hyperparameters can be modified to achive the desired result.

In [ ]:
close_to_grid = False
for i in range(20):
    graph, feat = generate_graph(model, n_nodes=18, feat_dim=40)
    G = nx.Graph(graph.cpu().to_networkx())
    G.remove_edges_from(nx.selfloop_edges(G))
    hist = nx.degree_histogram(G)
    correct_degrees = sum(hist[2:5])
    incorrec_degrees = sum(hist) - correct_degrees
    close_to_grid = nx.is_connected(G) and correct_degrees > 0 and incorrec_degrees == 0
    if close_to_grid: break
assert close_to_grid

### Task 7. Kolmogorov Smirnov distance score (1 point)

Let us check the model on real graphs. Let a quality metric be the closeness of generated graphs to real graphs. We will compute the sum of KS distances between distributions of statistics: clustering coefficients, laplacian spectrum values, node degrees.

Write a function `ks_score` that takes a list of real graphs, a list of generated (fake) graphs and computes the sum of KS distances. The input graphs are DGL graphs.

*Hint: use `ks_2samp`*

In [ ]:
def ks_score(fake_graphs, real_graphs):
    real_graphs = [nx.Graph(g.to_networkx()) for g in real_graphs]
    fake_graphs = [nx.Graph(g.to_networkx()) for g in fake_graphs]
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
rand_graph = dgl.from_networkx(nx.erdos_renyi_graph(10, 30))
grid55_graph = dgl.from_networkx(nx.grid_2d_graph(5, 5))
grid44_graph = dgl.from_networkx(nx.grid_2d_graph(4, 4))
score = ks_score([rand_graph], [grid55_graph])
assert score == 2.9
score = ks_score([grid55_graph], [grid55_graph])
assert score == 0
score = ks_score([grid55_graph], [grid44_graph])
assert score == 0.29

### Task 8. DFS from top degree node ordering (1 point)

So far, we have tested our model on grid graphs that have predefined node ordering. It helps us to train the model, but real networks are arbitrary ordered and so we need to select certain node ordering to train the model. There are many approaches: node degree, DFS/BFS from top node degree, k-core decomposition and so on. In this assignment, let us select DFS from top degree node.

Write a function `dfs_ordering` that takes a DGL graph and returns a tensor (array) of nodes where the first node is the top degree node and others are ordered by DFS from the top degree node. We assume that the graph has a single connected component.

In [ ]:
def dfs_ordering(graph):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = nx.complete_graph(5)
G.add_edges_from([[0, -1], [-1, -2], [-2, -3]])
graph = dgl.from_networkx(G)
ord = dfs_ordering(graph)
assert ord.shape == (8, )
adj = graph.adj().to_dense()
assert torch.all(adj[ord, :][:, ord][0] == torch.tensor([0, 1, 1, 1, 1, 1, 0, 0]))

### Task 9. Erdos-Renyi model baseline (1 point)

Create a simple baseline — Erdos-Renyi model.

Write a function `fit` that takes a list of DGL graphs and estimates the parameter $p$ of Erdos-Renyi model.

Write a function `sample` that generates an Erdos-Renyi graph with parameter $n$ and $p$. The output graph is DGL graph.

In [ ]:
class ErdosRenyi():
    def __init__(self):
        self.p = None
    def fit(self, graphs):
        # YOUR CODE HERE
        raise NotImplementedError()
    def sample(self, n):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
graphs = [dgl.rand_graph(10, 50), dgl.rand_graph(20, 100)]
er_model = ErdosRenyi()
er_model.fit(graphs)
assert er_model.p == (0.5 + 0.25) / 2
er_graph = er_model.sample(10)
assert 24 < er_graph.number_of_edges() < 51

### Task 10. Results on protein dataset (1 point)

In [ ]:
proteins_dataset = GINDataset('PROTEINS', self_loop=True)
len(proteins_dataset)

Let us test our model on proteins dataset. We will only learn small graphs for simplicity. Also we drops graph with multiple connected components.

In [ ]:
proteins_dataset = GINDataset('PROTEINS', self_loop=True)
small_graphs_idx = []
for i in range(len(proteins_dataset)):
    graph = proteins_dataset.graphs[i]
    if graph.number_of_nodes() <= 40:
        n_nodes = len(torch.cat(dgl.bfs_nodes_generator(graph, 0)))
        if graph.number_of_nodes() == n_nodes:
            small_graphs_idx.append(i)
proteins_dataset = Subset(proteins_dataset, small_graphs_idx)
len(proteins_dataset)

Let us split the dataset into a train and test sets. We will use the train set to train the model and use the test set to calculate KS distance score between real and fake graphs. Let the train ratio be approximately 0.3.

In [ ]:
torch.manual_seed(0)
train_mask = torch.rand(len(proteins_dataset)) < 0.3
train_idx = torch.where(train_mask)[0]
train_proteins_set = Subset(proteins_dataset, train_idx)
test_idx = torch.where(~train_mask)[0]
test_proteins_set = Subset(proteins_dataset, test_idx)
len(train_proteins_set), len(test_proteins_set)

Check that train and test are close with respect to KS distanse score.

In [ ]:
_train = [g for g, l in train_proteins_set]
_test = [g for g, l in test_proteins_set]
print(f'KS score: {ks_score(_train, _test):.4f}')

Also we want to learn distribution of number of nodes to generate graphs with different number of nodes.

In [ ]:
n_nodes_seq = [g.number_of_nodes() for g, l in proteins_dataset]
plt.hist(n_nodes_seq);

Generate graphs from Erdos Renyi model.

In [ ]:
fake_graphs = [er_model.sample(np.random.choice(n_nodes_seq)) for _ in range(100)]
print(f'KS score: {ks_score(fake_graphs, _test):.4f}')

Next, prepare the dataset for the GNN model (similar to the grid dataset).

In [ ]:
class GraphDataset(DGLDataset):
    def __init__(self, initial_set, feat_dim):
        self.initial_set = initial_set
        self.feat_dim = feat_dim
        self.graphs = None
        self.labels = None
        super().__init__(name='grid')

    def process(self):
        graphs = []
        labels = []
        for graph in self.initial_set:
            ord = dfs_ordering(graph)
            graph = dgl.reorder_graph(
                graph, 'custom', permute_config={'nodes_perm':ord})
            feat = torch.eye(graph.number_of_nodes())
            feat = F.pad(feat, [0, self.feat_dim - graph.number_of_nodes(), 0, 0])
            graph.ndata['feat'] = feat
            # YOUR CODE HERE
            raise NotImplementedError()
        self.graphs = graphs
        self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.labels[idx]

In [ ]:
graph_dataset = GraphDataset(_train, feat_dim=40)
assert len(graph_dataset) == 4701

In [ ]:
dataloader = DataLoader(
    graph_dataset, batch_size=512, collate_fn=collate, shuffle=True)
len(dataloader)

Train the model. To pass time limits, load the model parameters using loading Torch interface: https://pytorch.org/tutorials/beginner/saving_loading_models.html. There is also a way to speed the training process using iterations over blocks of nodes (details are in [Liao et al., 2019](https://arxiv.org/pdf/1910.00760.pdf)). In this assignment, the block size is 1 for simplicity.

In [ ]:
def train_on_proteins(dataloader):
    # YOUR CODE HERE
    raise NotImplementedError()
    # model = your model
    # url = link to parameters
    # open('params', 'wb').write(requests.get(url).content)
    # model.load_state_dict(torch.load('params'))
    # return model

    model = BernoulliGNN(in_dim=40, hid_dim=128, num_layer=5)
    model.to(device);
    opt = Adam(model.parameters(), lr=0.0005)
    n_epochs = 1000
    for e in trange(n_epochs):
        loss = train(model, dataloader, opt)
        if e % 50 == 0 or e+1 == n_epochs:
            print(f'Epoch: {e+1}/{n_epochs}, NLL loss: {loss:.4f}')
            plt.figure(figsize=(2,2))
            graph, feat = generate_graph(model, n_nodes=30, feat_dim=40)
            G = nx.Graph(graph.cpu().to_networkx())
            G.remove_edges_from(nx.selfloop_edges(G))
            nx.draw_kamada_kawai(G, node_size=30)
            plt.show()
    return model

In [ ]:
model = train_on_proteins(dataloader)
fake_graphs = []
for _ in range(100):
    graph = er_model.sample(np.random.choice(n_nodes_seq))
    fake_graphs.append(graph)
er_score = ks_score(fake_graphs, _test)

fake_graphs = []
for _ in trange(100):
    graph, feat = generate_graph(model, np.random.choice(n_nodes_seq), feat_dim)
    fake_graphs.append(graph.cpu())
gnn_score = ks_score(fake_graphs, _test)

assert gnn_score < er_score
assert gnn_score < 1

In [ ]:
print(f'Erdos Renyi KS score: {er_score:.4f}')
print(f'GNN model KS score: {gnn_score:.4f}')

Examples of real graphs

In [ ]:
plt.figure(figsize=(10, 10))
for j in range(9):
    plt.subplot(3, 3, j+1)
    graph = np.random.choice(_test)
    G = nx.Graph(graph.cpu().to_networkx())
    G.remove_edges_from(nx.selfloop_edges(G))
    nx.draw_kamada_kawai(G, node_size=30)

Examples of fake graphs

In [ ]:
plt.figure(figsize=(10, 10))
for j in range(9):
    plt.subplot(3, 3, j+1)
    graph, feat = generate_graph(model, n_nodes=np.random.choice(n_nodes_seq), feat_dim=40)
    G = nx.Graph(graph.cpu().to_networkx())
    G.remove_edges_from(nx.selfloop_edges(G))
    nx.draw_kamada_kawai(G, node_size=30)